In [1]:
import os
import torch
import pandas as pd
from sklearn.metrics.cluster import adjusted_rand_score,normalized_mutual_info_score,adjusted_mutual_info_score,silhouette_score
import numpy as np
import psutil,time,tracemalloc
##Note: To call the mclust clustering algorithm in the R package, make sure that mclust is installed in R and that rpy2 can be connected successfully
os.environ["R_HOME"] = r"D:\R-4.3.1"
os.environ["PATH"]   = r"D:\R-4.3.1\bin\x64" + ";" + os.environ["PATH"]


def eval_model(pred, labels=None):
    if labels is not None:
        label_df = pd.DataFrame({"True": labels, "Pred": pred}).dropna()
        # label_df = pd.DataFrame({"True": labels, "Pred": pred}).dropna()
        # completeness = completeness_score(label_df["True"], label_df["Pred"])
        # hm = homogeneity_score(label_df["True"], label_df["Pred"])
        ari = adjusted_rand_score(label_df["True"], label_df["Pred"])
        nmi = normalized_mutual_info_score(label_df["True"], label_df["Pred"])
        ami=adjusted_mutual_info_score(label_df["True"], label_df["Pred"])
    return  ari,nmi,ami

In [2]:

def run_GraphST(adata, dataset, random_seed = np.random.randint(100), device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') ):

    from GraphST_v1 import GraphST
    from utils import clustering

    model = GraphST.GraphST(adata, device=device,epochs=600, random_seed=random_seed)
    adata = model.train()
    n_clusters = len(set(adata.obs["ground_truth"].dropna()))
    # set radius to specify the number of neighbors considered during refinement
    radius = 50 # ### default radius = 50
    adata.obsm["embedding"] = adata.obsm["emb"]
    clustering(adata, n_clusters, radius=radius, method="mclust",refinement=False)
    ari, nmi, ami  = eval_model(adata.obs['domain'], adata.obs['ground_truth'])
    SC = silhouette_score(adata.obsm["embedding"], adata.obs['domain'])
    used_adata = adata[adata.obs["ground_truth"].notna()]

    clustering(adata, n_clusters, radius=radius, method="mclust", refinement=True)
    ari_r, nmi_r, ami_r = eval_model(adata.obs['domain'], adata.obs['ground_truth'])
    SC_r = silhouette_score(adata.obsm["embedding"], adata.obs['domain'])

    used_adata = adata[adata.obs["ground_truth"].notna()]

    res = {}
    res["dataset"] = dataset
    res["ari"] = ari
    res["nmi"] = nmi
    res["ami"] = ami
    res["sc"] = SC

    res["nmi_r"] = nmi_r
    res["ari_r"] = ari_r
    res["ami_r"] = ami_r
    res["sc_r"] = SC_r
    adata.obs["pred_label"] = adata.obs['domain']
    return res, adata


In [3]:
import sys
sys.path.append('../')
import utils_for_all as usa  ##Call a unified data reading function to avoid repetitive data input in each method
if __name__ == '__main__':

  Dataset=["ST_Hippocampus_2",'SlideV2_mouse_embryo_E8.5','151673',"SeqFish","STARmap","Stereo","Mouse_brain","Breast_cancer","PDAC"]
  Dataset_test=['151673']
for dataset in Dataset_test:
    if dataset.startswith('15'):
        save_data_path = f'../../Output/GraphST/DLPFC/{dataset}/'
    else:
        save_data_path = f'../../Output/GraphST/{dataset}/'
    if not os.path.exists( save_data_path):
        os.makedirs(save_data_path)

    adata, _ = usa.get_adata(dataset, data_path='../../Dataset/')
    adata.var_names_make_unique()

    best_ari = 0
    results=pd.DataFrame()
    for i in range(1):
        random_seed = 0
        start = time.time()
        tracemalloc.start()
        start_MB = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024

        res, adata_h5= run_GraphST(adata.copy(), dataset, random_seed=random_seed)

        end = time.time()
        end_MB = psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024
        uesd_time = end - start
        used_memo = end_MB - start_MB
        current, peak = tracemalloc.get_traced_memory()  #
        tracemalloc.stop()
        peak = peak / 1024.0 / 1024.0 / 1024.0
        print(u'Current memory usage_end:：%.4f GB' % used_memo)
        print('time: {:.4f} s'.format(uesd_time))
        print('memory blocks peak:{:>10.4f} GB'.format(peak))
        tracemalloc.clear_traces()

        res["time"] = uesd_time
        res["Memo"] = used_memo
        res["Memo_peak"] = peak
        res["round"] = i+1

        results = results._append(res, ignore_index=True)

    print(results.head())
    results.to_csv(save_data_path + "/{}_result.csv".format(dataset), header=True)
    adata_h5.write_h5ad(save_data_path + str(dataset) + ".h5ad")
    results.to_csv(f'{save_data_path}{dataset}_result.csv', header=True)
    results.set_index('dataset', inplace=True)
    print(results.head())
    res_mean = results.mean()
    res_mean.to_csv(f'{save_data_path}{dataset}_mean.csv', header=True)
    res_std = results.std()
    res_std.to_csv(f'{save_data_path}{dataset}_std.csv', header=True)
    res_median = results.median()
    res_median.to_csv(f'{save_data_path}{dataset}_median.csv', header=True)

Current memory usage_end:：0.0000 GB
time: 0.0000 s
memory blocks peak:    0.0000 GB
load DLPFC dataset:


D:\Anaconda_install\envs\STAGATE\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
D:\Anaconda_install\envs\STAGATE\lib\site-packages\ot\backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


Edge count： 10917
Average number of spot： 3.0
get Top2000HVG genes，Zero radio： 0.8409240175872492
Begin to train ST data...


100%|██████████| 600/600 [00:21<00:00, 28.49it/s]


Optimization finished for ST data!
begin mclust clustering


R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.0.0
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%
mclust finished： 4.411609888076782
begin mclust clustering
fitting ...
  |======================================================================| 100%
mclust finished： 3.156996488571167
Current memory usage_end:：0.3446 GB
time: 42.4082 s
memory blocks peak:    0.9602 GB
  dataset       ari       nmi       ami        sc     nmi_r    ari_r  \
0  151673  0.590827  0.698313  0.697499  0.082565  0.727536  0.62527   

      ami_r      sc_r       time      Memo  Memo_peak  round  
0  0.726798  0.072619  42.408249  0.344593   0.960152      1  
              ari       nmi       ami        sc     nmi_r    ari_r     ami_r  \
dataset                                                                        
151673   0.590827  0.698313  0.697499  0.082565  0.727536  0.62527  0.726798   

             sc_r       time      Memo  Memo_peak  round  
dataset                                                   
151673 